In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

import instrumental_programs as iv

import scipy.optimize as opt

import matplotlib.pyplot as plt
import seaborn as sns

import sys
from iv_lp import *

import pickle as pk

from tqdm.notebook import tqdm

%matplotlib inline

In [ ]:
########################################################
# Data location (change this)
########################################################

data_path = "../data/OHIE_Public_Use_Files/OHIE_Data/"

########################################################

# Load data

In [ ]:
dv = pd.read_stata(f"{data_path}oregonhie_descriptive_vars.dta")
print(dv.shape)
sp = pd.read_stata(f"{data_path}oregonhie_stateprograms_vars.dta")
print(sp.shape)
ip = pd.read_stata(f"{data_path}oregonhie_inperson_vars.dta")
print(ip.shape)
data = dv.merge(sp).merge(ip)
print(data.shape)
data = data[data.sample_resp_inp == "Respondent"]
print(data.shape)


# has depression
data["has_depression"] = 1*(data.phqtot_inp >= 10)
data["phq_cat"] = (data.phqtot_inp.values/5).astype(int)

# enrolled in Medicaid
data["ohp_enrolled"] = 1*(data.ohp_all_ever_inperson == "Enrolled")

In [ ]:
# plot PHQ-9 distribution
sns.displot(ip.phqtot_inp)
plt.show()

### Count members of each household that enrolled in the lottery

In [ ]:
household_counts = data.household_id.value_counts().reset_index()
household_counts.rename({"household_id":"household_count"},axis=1,inplace=True)
household_counts.rename({"index":"household_id"},axis=1,inplace=True)
data = data.merge(household_counts,how="left",on="household_id")
data.shape

### Get dummies for household count

In [ ]:
data["household_count_orig"] = data["household_count"]
data = pd.get_dummies(data,columns=["household_count"],drop_first=True)

### Filter data to include on participants with a PHQ-9 score and single family households

In [ ]:
# has phq score
mmask = ~data.phqtot_inp.isna()
# has household size = 1
cmask = data.household_count_orig == 1

masked_data = data[mmask&cmask].copy()

# ITT w/ measurement error

In [ ]:
def cost_fun(v):
    return 1*(v < 3)

### Estimate the observed data distribution P(lottery, medicaid, depression)

In [ ]:
Z = 1*(masked_data["treatment"] == "Selected").values
T = masked_data["ohp_enrolled"].values
M = masked_data["phq_cat"].values
M = np.max(M) - M

# Survey weights from Baicker et al. (2018)
weights = masked_data.weight_total_inp.values.copy()

# Weighted empirical dist
pztm = np.zeros((2,2,5))
for z in range(2):
    for t in range(2):
        for m in range(5):
            pztm[z,t,m] = np.average((Z==z)&(M==m)&(T==t),weights=weights)
            

# Calculate effective sample sizes
sample_sizes = np.zeros(2)
for z in range(2):
    sample_sizes[z] = np.sum(weights[Z==z])**2/np.sum(weights[Z==z]**2)
    

In [ ]:
# Define measurement error models
assumptions = [
    {"EB1":True,
     "EB2":False,
     "Mon":False,
     "Sym":False,
     "Exp":False},
    
    {"EB1":True,
     "EB2":False,
     "Mon":False,
     "Sym":False,
     "Exp":True},
    
    {"EB1":True,
     "EB2":False,
     "Mon":True,
     "Sym":False,
     "Exp":True},
    
    {"EB1":True,
     "EB2":False,
     "Mon":False,
     "Sym":True,
     "Exp":True},
]


# Define epsilon grid
S = 41
grid_epsilons = np.linspace(0,0.04,S)

# Empty arrays for point estimated bounds and confidence intervals
grid_point_bounds = np.zeros((len(assumptions),S,2))
grid_ci_bounds = np.zeros((len(assumptions),S,2))
            
for k,ass in enumerate(assumptions):
    print(ass)
    
    for s in tqdm(list(range(len(grid_epsilons)))):
        eps = grid_epsilons[s]

        sym_const = ass["Sym"]
        mon_const = ass["Mon"]
        exp_const = ass["Exp"]
        pos_effect_of_truth = False
        pos_effect_of_treatment = True

        eb_args = []
        if ass["EB1"]:
            eb_args.append((0.0,eps))
        if ass["EB2"]:
            eb_args.append((1.0,eps/10))

        lb,ub,lb_res,ub_res,lp_args = get_iv_bounds(pztm,
                                    0,
                                    eb_args,
                                    err_const=True,
                                    mon_const=mon_const,
                                    sym_const=sym_const,
                                    exp_const=exp_const,
                                    pos_effect_of_treatment=pos_effect_of_treatment,
                                    pos_effect_of_truth=False,
                                    parameter="ate",
                                    alpha=0.99999999,
                                    n=100000000,
                                    fun=cost_fun,
                                    x0=None)

        grid_point_bounds[k,s,0] = lb
        grid_point_bounds[k,s,1] = ub
        
        lb,ub,lb_res,ub_res,lp_args = get_iv_bounds(pztm,
                                    0,
                                    eb_args,
                                    err_const=True,
                                    mon_const=mon_const,
                                    sym_const=sym_const,
                                    exp_const=exp_const,
                                    pos_effect_of_treatment=pos_effect_of_treatment,
                                    pos_effect_of_truth=False,
                                    parameter="ate",
                                    alpha=0.05,
                                    n=None,
                                    fun=cost_fun,
                                    x0=None,
                                    sample_sizes=sample_sizes)

        grid_ci_bounds[k,s,0] = lb
        grid_ci_bounds[k,s,1] = ub

In [ ]:
line_styles = ["k--","k:","k-.","k"]
labels = ["EB only","EB+Exp","EB+Exp+Mon","EB+Exp+Sym"]

plt.figure(figsize=(9,6))

for k in range(grid_point_bounds.shape[0]):
    plt.plot(grid_epsilons,grid_point_bounds[k,:,0],line_styles[k],label=labels[k])
    plt.plot(grid_epsilons,grid_point_bounds[k,:,1],line_styles[k])

plt.fill_between(grid_epsilons,grid_point_bounds[0,:,0],grid_point_bounds[0,:,1],color="C0",alpha=0.25)
plt.fill_between(grid_epsilons,grid_ci_bounds[0,:,0],grid_ci_bounds[0,:,1],color="C0",alpha=0.1)
plt.grid()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel(r"Error budget ($\epsilon$)",fontsize=18)
plt.ylabel("ATE",fontsize=18)
plt.legend(loc="center right",fontsize=14,framealpha=1.0)
# plt.savefig("../output/figures/ohie_sensitivity_grid_plot.png",dpi=250)
print(plt.gca().get_ylim())
plt.show()

In [ ]:
# Calculate maximum error budgets
for k in range(grid_point_bounds.shape[0]):
    print(f"{labels[k]}: {grid_epsilons[np.argmax(grid_point_bounds[k,:,1] > -1e-5)]}")